<a href="https://colab.research.google.com/github/bhuvanmdev/raw-nn-notebooks/blob/main/notebooks/ANN_MNSIT_and_MAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !mkdir -p ~/.kaggle
# !cp /content/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d heptapod/titanic
# !unzip titanic.zip

Archive:  titanic.zip
  inflating: train_and_test2.csv     


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(r'train_and_test2.csv')
data.dropna(inplace=True,axis='rows')
print(data.columns)
data.drop({'Passengerid','zero','zero.1','zero.2','zero.3','zero.4','zero.5','zero.6','zero.7','zero.8','zero.9','zero.10','zero.11','zero.12','sibsp',
              'zero.13','zero.14','zero.15','zero.16','zero.17','zero.18',},inplace=True,axis=1)
print(data.columns)
print(k:=data["Embarked"].unique())
data["Embarked"].replace(to_replace={x:y for y,x in enumerate(k)},inplace=True)
print(str(data).count('nan'))
# data["Sex"].replace({'male':0,'female':1},inplace=True)
data["Fare"] /= 1e3
y = data["2urvived"]
data.drop('2urvived',inplace=True,axis='columns')
print(y)
xtrain,xtest,ytrain,ytest = train_test_split(data,y,train_size=0.75,random_state=43)

Index(['Passengerid', 'Age', 'Fare', 'Sex', 'sibsp', 'zero', 'zero.1',
       'zero.2', 'zero.3', 'zero.4', 'zero.5', 'zero.6', 'Parch', 'zero.7',
       'zero.8', 'zero.9', 'zero.10', 'zero.11', 'zero.12', 'zero.13',
       'zero.14', 'Pclass', 'zero.15', 'zero.16', 'Embarked', 'zero.17',
       'zero.18', '2urvived'],
      dtype='object')
Index(['Age', 'Fare', 'Sex', 'Parch', 'Pclass', 'Embarked', '2urvived'], dtype='object')
[2. 0. 1.]
0
0       0
1       1
2       1
3       1
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: 2urvived, Length: 1307, dtype: int64


In [ ]:
print(xtrain.shape,ytrain.shape,xtest.shape,ytest.shape)
ytrain = np.expand_dims(ytrain,axis=1)
print(xtrain.shape,ytrain.shape,xtest.shape,ytest.shape)
# ytrain = np.reshape(ytrain,981)

(980, 6) (980,) (327, 6) (327,)
(980, 6) (980, 1) (327, 6) (327,)


In [ ]:
class Model:
    def __init__(self,inp,out,epoch,alpha=0.1):
        self.a0 = inp
        self.batch = self.a0.shape[1]
        self.inp = inp
        np.array((10,1))
        self.out =
        self.epoch = epoch
        self.alpha = alpha
        self.w1 = np.random.randn(10,6)/3
        self.b1 = np.random.randn(10,1)
        self.w2 = np.random.randn(10,10)/3
        self.b2 = np.random.randn(10,1)
        self.w3 = np.random.randn(1,10)/3
        self.b3 = np.random.randn(10,1)

    def relu(self,z):
        return np.maximum(0,z)

    def softmax(self,z):
        return np.exp(z)/np.sum(np.exp(z))

    def front(self):
        self.z1 = self.w1.dot(self.a0) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = self.w2.dot(self.a1) + self.b2
        self.a2 = self.relu(self.z2)
        self.z3 = self.w3.dot(self.a2) + self.b3
        self.a3 = self.softmax(self.z3)

    def derivative_ReLU(self,z):
        return z > 0

    def back(self):
        self.dz3 = self.a3 - self.out
        self.dw3 = 1/self.batch * (self.dz3.dot(self.a2.T))
        self.db3 = 1/self.batch * np.sum(self.dz3, axis=1)#sum the rows

        self.dz2 = self.w3.T.dot(self.dz3) * self.derivative_ReLU(self.z2)
        self.dw2 = 1/self.batch * (self.dz2.dot(self.a1.T))
        self.db2 = 1/self.batch * np.sum(self.dz2, axis=1)

        self.dz1 = self.w2.T.dot(self.dz2) * self.derivative_ReLU(self.z1)
        self.dw1 = 1/self.batch * (self.dz1.dot(self.a0.T))
        self.db1 = 1/self.batch * np.sum(self.dz1, axis=1)

    def update(self):
        self.w1 -= self.alpha*self.dw1
        self.b1 -= self.alpha*np.reshape(self.db1,(10,1))
        self.w2 -= self.alpha*self.dw2
        self.b2 -= self.alpha*np.reshape(self.db2,(10,1))
        self.w3 -= self.alpha*self.dw3
        self.b3 -= self.alpha*np.reshape(self.db3,(10,1))

    def predict(self):
        return np.sum(np.argmax(self.a3,axis=0)==self.out)/self.out.size

    def gradient_descent(self):
        # size , m = self.a0.shape

        for i in range(self.epoch):
            self.front()
            self.back()
            self.update()

            if (i+1) % int(self.epoch/10) == 0:
                print(f"Iteration: {i+1} / {self.epoch}")
                print(f'{round(self.predict(),2)}')

k = Model(xtrain,ytest,10)
k.gradient_descent()

ValueError: ignored

In [ ]:
import numpy as np

class Model:
    def __init__(self, inp, out, epoch, alpha=1e-6):
        self.a0 = inp
        self.batch = self.a0.shape[1]
        self.inp = inp
        self.out = np.reshape(out, (1, out.shape[0]))  # Reshape to (1, 900)
        self.epoch = epoch
        self.alpha = alpha
        self.w1 = np.random.randn(100, 6)  /3 # Adjusted shape
        self.b1 = np.random.randn(100, 1)/3
        self.w2 = np.random.randn(100, 100)/3   # Adjusted shape
        self.b2 = np.random.randn(100, 1)/3
        self.w3 = np.random.randn(1, 100)/3
        self.b3 = np.random.randn(1, 1) /3 # Adjusted shape

    def relu(self, z):
        return np.maximum(0, z)

    def softmax(self, z):
        # if str()
        exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
          # Subtracting max for stability
        return exp_z / np.sum(exp_z, axis=0)

    def front(self):
        self.z1 = self.w1.dot(self.a0) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = self.w2.dot(self.a1) + self.b2
        self.a2 = self.relu(self.z2)
        self.z3 = self.w3.dot(self.a2) + self.b3
        # print(self.z3)
        # input(    )
        self.a3 = self.softmax(self.z3)

    def derivative_ReLU(self, z):
        return z > 0

    def back(self):
        self.dz3 = 2*(self.a3 - self.out)
        self.dw3 = 1 / self.batch * (self.dz3.dot(self.a2.T))
        self.db3 = 1 / self.batch * np.sum(self.dz3, axis=1)  # sum along rows

        self.dz2 = self.w3.T.dot(self.dz3) * self.derivative_ReLU(self.z2)
        self.dw2 = 1 / self.batch * (self.dz2.dot(self.a1.T))
        self.db2 = 1 / self.batch * np.sum(self.dz2, axis=1)

        self.dz1 = self.w2.T.dot(self.dz2) * self.derivative_ReLU(self.z1)
        self.dw1 = 1 / self.batch * (self.dz1.dot(self.a0.T))
        self.db1 = 1 / self.batch * np.sum(self.dz1, axis=1)

    def update(self):
        self.w1 -= self.alpha*self.dw1
        self.b1 -= self.alpha*np.reshape(self.db1,(100,1))
        self.w2 -= self.alpha*self.dw2
        self.b2 -= self.alpha*np.reshape(self.db2,(100,1))
        self.w3 -= self.alpha*self.dw3
        self.b3 -= self.alpha*np.reshape(self.db3,(1,1))

    def predict(self):
        predictions = np.argmax(self.a3, axis=0)
        accuracy = np.sum(predictions == self.out) / self.out.size
        return accuracy

    def gradient_descent(self):
        for i in range(self.epoch):
            self.front()
            self.back()
            self.update()

            if (i + 1) % int(self.epoch / 10) == 0:
                print(f"Iteration: {i+1} / {self.epoch}")
                print(f'Accuracy: {round(self.predict() * 100, 2)}%')

# xtrain = np.random.randn(10, 900)  # Example data
# ytest = np.random.randint(0, 10, size=900)  # Example labels
k = Model(xtrain.T, ytrain, 1000)
k.gradient_descent()


Iteration: 100 / 1000
Accuracy: 74.18%
Iteration: 200 / 1000
Accuracy: 74.18%
Iteration: 300 / 1000
Accuracy: 74.18%
Iteration: 400 / 1000
Accuracy: 74.18%
Iteration: 500 / 1000
Accuracy: 74.18%
Iteration: 600 / 1000
Accuracy: 74.18%
Iteration: 700 / 1000
Accuracy: 74.18%
Iteration: 800 / 1000
Accuracy: 74.18%
Iteration: 900 / 1000
Accuracy: 74.18%
Iteration: 1000 / 1000
Accuracy: 74.18%


# New Section

In [ ]:
def expand_dims_1_to_4(tensor, dims=None):
    if not dims:
        dims = [-1, -1, -1]
    return tf.expand_dims(
             tf.expand_dims(
               tf.expand_dims(tensor, axis=dims[0]),
               axis=dims[1]),
             axis=dims[2])

def res_manipulator(enc_a,
                    enc_b,
                    amplification_factor,
                    layer_dims=32,
                    num_resblk=1,
                    num_conv=0,
                    num_aft_conv=0,
                    probe_pt=None):
    diff = (enc_b - enc_a)
    if probe_pt is not None:
        probe_pt["mani_diff"] = diff
    for i in range(num_conv):
        p = 3
        k = 7
        diff = tf.pad(diff, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
        cname = f'mani_conv_{i}'
        diff = Conv2D(layer_dims, kernel_size=k, strides=1, activation='relu', name=cname + 'c')(diff)
    if probe_pt is not None:
        probe_pt["mani_after_conv"] = diff
    diff = diff * expand_dims_1_to_4(amplification_factor - 1.0)
    if probe_pt is not None:
        probe_pt["mani_after_mult"] = diff
    for i in range(num_aft_conv):
        diff = tf.pad(diff, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
        cname = f'mani_aft_conv_{i}'
        diff = Conv2D(layer_dims, kernel_size=3, strides=1, name=cname + 'c')(diff)
    for i in range(num_resblk):
        diff = residual_block(diff, layer_dims, 3, 1, name=f'mani_resblk{i}')
    if probe_pt is not None:
        probe_pt["mani_after_res"] = diff
    return enc_b + diff




def res_encoder(image, no,layer_dims=32, num_resblk=5):
    c0 = tf.pad(image, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
    c1 = Conv2D(layer_dims // 2, kernel_size=7, strides=1,activation='relu', name='res_enc_conv1_c'+no)(c0)
    c2 = Conv2D(layer_dims, kernel_size=3, strides=2,activation='relu', name='res_enc_conv2_c'+no)(c1)
    # Define G network with num_resblk resnet blocks
    r = c2
    for i in range(num_resblk):
        r = residual_block(r, layer_dims, 3, 1, name=f'res_encoder_resblk{i}_'+no)

    return r




def res_decoder(activation,
                layer_dims=64,
                out_channels=3,
                num_resblk=4):
    r = activation
    for i in range(num_resblk):
        r = residual_block(r, layer_dims, 3, 1, name=f'res_decoder_resblk{i}')

    up = tf.image.resize(r, tf.shape(r)[1:3] * 2)
    up = tf.pad(up, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
    d2 = Conv2D(layer_dims // 2, kernel_size=3, strides=1, activation='relu',name='res_dec_conv2_c')(up)
    d2 = tf.pad(d2, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
    out = Conv2D(out_channels, kernel_size=7, strides=1, name='res_pred_conv')(d2)

    return out

# Define the residual_block function



def residual_block(x, output_dim, ks=3, s=1, name='residual_block'):
    p = (ks - 1) // 2
    y = tf.pad(x, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
    y = Conv2D(output_dim, ks, s, activation='relu',name=name+'_c1')(y)
    y = tf.pad(y, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
    y = Conv2D(output_dim, ks, s, name=name+'_c2')(y)
    return y + x

def _encoder(image,no):
    enc = res_encoder(image,no)

    texture_enc = enc
    shape_enc = enc
    # first convolution on common encoding
    if use_texture_conv:
        stride = 2 if texture_downsample else 1###making sure it is downsampled or not
        texture_enc = Conv2D(32, 3, stride,activation='relu',name='enc_texture_conv_'+no)(texture_enc)

    if use_shape_conv:
        shape_enc = Conv2D(32, 3, 1,activation='relu', name='enc_shape_conv_'+no)(shape_enc)

    for i in range(1):
        name = f'texture_enc_{i}_'+no
        if i == 0:
            # for backward compatibility
            name = 'texture_enc__'+no
        texture_enc = residual_block(texture_enc, 32, 3, 1,'resblk_'+name)

    for i in range(1):
        name = f'shape_enc_{i}_'+no
        if i == 0:
            # for backward compatibility
            name = 'shape_enc__'+no
        shape_enc = residual_block(shape_enc, 32,
                                   3, 1, 'resblk_'+name)
    return texture_enc, shape_enc

def _decoder(texture_enc, shape_enc):
    if texture_downsample:
        texture_enc = tf.image.resize(
                        texture_enc,
                        tf.shape(texture_enc)[1:3]* 2)
        texture_enc = tf.pad(texture_enc, [[0, 0], [1, 1], [1, 1], [0, 0]],
                             "REFLECT")
        texture_enc = Conv2D(32,3, 1,activation='relu',name='texture_upsample')(texture_enc)

    enc = tf.concat([texture_enc, shape_enc], axis=3)
    # Needs double the channel because we concat the two encodings.
    return res_decoder(enc)

In [ ]:
use_texture_conv = False
use_shape_conv = False
texture_downsample = False
probe_pt = {}

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Input,Layer
from tensorflow.nn import relu
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import TruncatedNormal
import pydot,graphviz
from IPython.display import Image

In [ ]:
def expand_dims_1_to_4(tensor, dims=None):
    if not dims:
        dims = [-1, -1, -1]
    return tf.expand_dims(
               tf.expand_dims(tensor, axis=dims[0]),
               axis=dims[1])
def expand_dims_1_to_4_np(arr, dims=None):
    if not dims:
        dims = [-1, -1, -1]
    return np.expand_dims(
             np.expand_dims(
               np.expand_dims(arr, axis=dims[0]),
               axis=dims[1]),
             axis=dims[2])
def res_manipulator(enc_a,
                        enc_b,
                        amplification_factor=10,
                        layer_dims=32,
                        num_resblk=1,
                        num_conv=0,
                        num_aft_conv=0,
                        probe_pt=probe_pt):
    diff = (enc_b - enc_a)
    if probe_pt is not None:
        probe_pt["mani_diff"] = diff
    for i in range(num_conv):
        p = 3
        k = 7
        diff = tf.pad(diff, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
        cname = f'mani_conv_{i}'
        diff = Conv2D(layer_dims, kernel_size=k, strides=1, activation='relu', name=cname + 'c',kernel_initializer=TruncatedNormal(stddev=0.2))(diff)
    if probe_pt is not None:
        probe_pt["mani_after_conv"] = diff
    k = expand_dims_1_to_4(amplification_factor - 1.0)
    print(k.shape)
    diff = diff *k
    if probe_pt is not None:
        probe_pt["mani_after_mult"] = diff
    for i in range(num_aft_conv):
        diff = tf.pad(diff, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
        cname = f'mani_aft_conv_{i}'
        diff = Conv2D(layer_dims, kernel_size=3, strides=1, name=cname + 'c',kernel_initializer=TruncatedNormal(stddev=0.2))(diff)
    for i in range(num_resblk):
        diff = residual_block(diff, layer_dims, 3, 1, name=f'mani_resblk{i}')
    if probe_pt is not None:
        probe_pt["mani_after_res"] = diff
    return enc_b + diff

layer_dims=32
convc1 = Conv2D(layer_dims // 2, kernel_size=7, strides=1,activation='relu',name='ds', kernel_initializer=TruncatedNormal(stddev=0.2))
convc2 = Conv2D(layer_dims, kernel_size=3, strides=2,activation='relu', kernel_initializer=TruncatedNormal(stddev=0.2))

class rb:
    def __init__(self,output_dim,ks,s,no_blk,tex_dim,shp_dim,num_shp_res,num_tex_res):
        self.tex_dim = tex_dim
        self.shp_dim = shp_dim
        self.output_dim = self.layer_dims = output_dim
        self.num_tex_res = num_tex_res
        self.num_shp_res = num_shp_res
        self.no_blk = no_blk
        self.res_enc_conv = [Conv2D(self.layer_dims // 2, kernel_size=7, strides=1,activation='relu',name='ds', kernel_initializer=TruncatedNormal(stddev=0.2))
                            ,Conv2D(self.layer_dims, kernel_size=3, strides=2,activation='relu', kernel_initializer=TruncatedNormal(stddev=0.2))]
        self.resblk_res = [(Conv2D(self.output_dim, ks, s, activation='relu',kernel_initializer=TruncatedNormal(stddev=0.2)),
                            Conv2D(self.output_dim, ks, s,kernel_initializer=TruncatedNormal(stddev=0.2))) for _ in range(self.no_blk)]
        self.tex_resblk = [(Conv2D(self.tex_dim, ks, s, activation='relu',kernel_initializer=TruncatedNormal(stddev=0.2)),
                                Conv2D(self.tex_dim, ks, s,kernel_initializer=TruncatedNormal(stddev=0.2))) for _ in range(self.num_tex_res)]
        self.shp_resblk = [(Conv2D(self.shp_dim, ks, s, activation='relu',kernel_initializer=TruncatedNormal(stddev=0.2))
                                ,Conv2D(self.shp_dim, ks, s,kernel_initializer=TruncatedNormal(stddev=0.2))) for _ in range(self.num_shp_res)]
        self.tex_conv,self.shp_conv = None,None

    def residual_block(self,lis,x, output_cov_blk, ks=3, s=1, name='residual_block'):
        p = (ks - 1) // 2
        y = tf.pad(x, [[0,0], [p,p], [p,p], [0,0]], 'REFLECT')
        y = lis[output_cov_blk][0](y)
        y = tf.pad(y, [[0,0], [p,p], [p,p], [0,0]], 'REFLECT')
        y = lis[output_cov_blk][1](y)
        return y + x

    def res_encoder(self,image, no, num_resblk=1):
        c0 = tf.pad(image, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
        c1 = self.res_enc_conv[0](c0)
        c2 = self.res_enc_conv[1](c1)
        # Define G network with num_resblk resnet blocks#name='res_enc_conv1_c'+no,name='res_enc_conv2_c'+no,
        r = c2
        for i in range(num_resblk):
            r = self.residual_block(self.resblk_res,r, i, 3, 1, name=f'res_encoder_resblk{i}_'+no)
        return r

    def _encoder(self,image,no):
        enc = self.res_encoder(image,no)
        texture_enc = enc
        shape_enc = enc
        # first convolution on common encoding
        if use_texture_conv:
            stride = 2 if texture_downsample else 1
            if self.tex_conv==None:
              self.tex_conv = Conv2D(32, 3, stride,activation='relu',name='enc_texture_conv_1',kernel_initializer=TruncatedNormal(stddev=0.2))
            texture_enc = self.tex_conv(texture_enc)

        if use_shape_conv:
            if self.shp_conv == None:
                      self.shp_conv = Conv2D(32, 3, 1,activation='relu', name='enc_shape_conv_1',kernel_initializer=TruncatedNormal(stddev=0.2))###making sure it is downsampled or not
            shape_enc = self.shp_conv(shape_enc)

        for i in range(1):
            name = f'texture_enc_{i}_'
            if i == 0:
                # for backward compatiility
                name = 'texture_enc__'
            texture_enc = self.residual_block(self.tex_resblk,texture_enc, i, 3, 1,'resblk_'+name)

        for i in range(1):
            name = f'shape_enc_{i}_'+no
            if i == 0:
                # for backward compatibility
                name = 'shape_enc__'+no
            shape_enc = self.residual_block(self.shp_resblk,shape_enc, i,
                                       3, 1, 'resblk_'+name)
        return texture_enc, shape_enc

    def res_decoder(activation,
                    layer_dims=64,
                    out_channels=3,
                    num_resblk=1):
        r = activation
        for i in range(num_resblk):
            r = residual_block(r, layer_dims, 3, 1, name=f'res_decoder_resblk{i}')

        up = tf.image.resize(r, tf.shape(r)[1:3] * 2)
        up = tf.pad(up, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
        d2 = Conv2D(layer_dims // 2, kernel_size=3, strides=1, activation='relu',name='res_dec_conv2_c',kernel_initializer=TruncatedNormal(stddev=0.2))(up)
        d2 = tf.pad(d2, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
        out = Conv2D(out_channels, kernel_size=7, strides=1, name='res_pred_conv',kernel_initializer=TruncatedNormal(stddev=0.2))(d2)
        #i put this
        return tf.pad(out, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")


    def _decoder(self,texture_enc, shape_enc):
        if texture_downsample:
            texture_enc = tf.image.resize(
                            texture_enc,
                            tf.shape(texture_enc)[1:3]* 2)
            texture_enc = tf.pad(texture_enc, [[0, 0], [1, 1], [1, 1], [0, 0]],
                                 "REFLECT")
            texture_enc = Conv2D(32,3, 1,activation='relu',name='texture_upsample',kernel_initializer=TruncatedNormal(stddev=0.2))(texture_enc)

        enc = tf.concat([texture_enc, shape_enc], axis=3)
        # Needs double the channel because we concat the two encodings.
        return self.res_decoder(enc)

class Custom(Layer):
    def __init__(self,var,t_a,s_b):
        # super(Custom,self).__init__(self,var,t_a,s_b)
        super().__init__()
        self.var = var
        self.t_a = t_a
        self.s_b = s_b


    def call(self,img):
        texture_c, shape_c = self.var._encoder(img,'h')
        return 0.01*tf.reduce_mean(tf.abs(self.t_a - texture_c))+0.01*tf.reduce_mean(tf.abs(self.s_b - shape_c))
        # print(12)

res = rb(32,ks=3,s=1,
         no_blk=1,num_shp_res=1,num_tex_res=1,
         tex_dim=32,shp_dim=32)

def res_encoder(image, no,layer_dims=32, num_resblk=1):
    c0 = tf.pad(image, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
    c1 = convc1(c0)
    c2 = convc2(c1)
    # Define G network with num_resblk resnet blocks#name='res_enc_conv1_c'+no,name='res_enc_conv2_c'+no,
    r = c2
    for i in range(num_resblk):
        r = res.residual_block(r, i, 3, 1, name=f'res_encoder_resblk{i}_'+no)

    return r
def res_decoder(activation,
                    layer_dims=64,
                    out_channels=3,
                    num_resblk=1):
    r = activation
    for i in range(num_resblk):
        r = residual_block(r, layer_dims, 3, 1, name=f'res_decoder_resblk{i}')

    up = tf.image.resize(r, tf.shape(r)[1:3] * 2)
    up = tf.pad(up, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
    d2 = Conv2D(layer_dims // 2, kernel_size=3, strides=1, activation='relu',name='res_dec_conv2_c',kernel_initializer=TruncatedNormal(stddev=0.2))(up)
    d2 = tf.pad(d2, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
    out = Conv2D(out_channels, kernel_size=7, strides=1, name='res_pred_conv',kernel_initializer=TruncatedNormal(stddev=0.2))(d2)
    #i put this
    return tf.pad(out, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")

    # Define the residual_block function


#name=name+'_c1' , name=name+'_c2',
def residual_block(x, output_dim, ks=3, s=1, name='residual_block'):
    p = (ks - 1) // 2
    y = tf.pad(x, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
    y = Conv2D(output_dim, ks, s, activation='relu',kernel_initializer=TruncatedNormal(stddev=0.2))(y)
    y = tf.pad(y, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
    y = Conv2D(output_dim, ks, s, kernel_initializer=TruncatedNormal(stddev=0.2))(y)
    return y + x

#with tf.variable_scope('encoder', reuse=True)
class Mymodel(Model):
    def __init__(self,img_a,img_b,img_c,alpha):
        # super(Mymodel,self).__init__(self,img_a,img_b,img_c,alpha)
        self.t_a,self.s_a = res._encoder(img_a,'2')
        self.t_b,self.s_b = res._encoder(img_b,'2')
        self.custom_loss = Custom(res,self.t_a,self.s_b).call(img_c)
        self.ou = res_manipulator(self.s_b,self.s_a,amplification)#tf.constant([10],dtype="float32")
        out = self._decoder(self.t_b,self.ou)
        super().__init__(inputs=[img_a,img_b,img_c,alpha],outputs=out)
        super().add_loss(self.custom_loss)
    def _decoder(self,texture_enc, shape_enc):
        if texture_downsample:
            texture_enc = tf.image.resize(
                            texture_enc,
                            tf.shape(texture_enc)[1:3]* 2)
            texture_enc = tf.pad(texture_enc, [[0, 0], [1, 1], [1, 1], [0, 0]],
                                 "REFLECT")
            texture_enc = Conv2D(32,3, 1,activation='relu',name='texture_upsample',kernel_initializer=TruncatedNormal(stddev=0.2))(texture_enc)

        enc = tf.concat([texture_enc, shape_enc], axis=3)
        # Needs double the channel because we concat the two encodings.
        return res_decoder(enc)



In [ ]:
# img_a = Input((450,450,3))
# img_b = Input((450,450,3))
# _,s_a = _encoder(img_a,'1')
# t_b,s_b = _encoder(img_b,'2')
# ou = res_manipulator(s_b,s_a,10)
# out = _decoder(t_b,ou)
# model = Model(inputs=[img_a,img_b],outputs=out)

In [ ]:
img_a = Input((450,450,3),dtype="float32")
img_b = Input((450,450,3),dtype="float32")
img_c = Input((450,450,3),dtype="float32")
amplification = Input((1),dtype="float32")

# _,s_a = _encoder(img_a,'2')

# t_b,s_b = _encoder(img_b,'2')
# ou = res_manipulator(s_b,s_a,amplification)#tf.constant([10],dtype="float32")
# out = _decoder(t_b,ou)
model = Mymodel(img_a,img_b,img_c,amplification)


(None, 1, 1, 1)


In [ ]:
model.summary()
model.compile(optimizer='adam', loss=(lambda x,y:tf.reduce_mean(tf.abs(x - y))), metrics=['accuracy'])
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_checkpoint.h5',
    monitor='accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

Model: "mymodel_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 450, 450, 3)]        0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 450, 450, 3)]        0         []                            
                                                                                                  
 tf.compat.v1.pad_65 (TFOpL  (None, 456, 456, 3)          0         ['input_6[0][0]']             
 ambda)                                                                                           
                                                                                                  
 tf.compat.v1.pad_58 (TFOpL  (None, 456, 456, 3)          0         ['input_5[0][0]']     

In [ ]:
# model.summary()
model.compile(optimizer='adam', loss=(lambda x,y:tf.reduce_mean(tf.abs(x - y))), metrics=['accuracy'])

In [ ]:
!mkdir /content/mk
!ffmpeg -i baby.mp4 -r 5 /content/mk/%0d.png

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
# plot_model(model,'/content/model_box.jpeg',show_shapes=False,
#     show_dtype=True,
#     show_layer_names=True,
#     rankdir='LR',
#     dpi=1000,
#     expand_nested=True,
#     show_layer_activations=False,
#     show_trainable=True)
# Image('model_box.jpeg')
!zip model.zip model_box.jpeg

  adding: model_box.jpeg (deflated 64%)


In [ ]:
!git clone https://github.com/flyingzhao/PyEVM.git

Cloning into 'PyEVM'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 112 (delta 27), reused 27 (delta 27), pack-reused 83
Receiving objects: 100% (112/112), 111.06 MiB | 40.07 MiB/s, done.
Resolving deltas: 100% (57/57), done.
